<H1>AML Project 2: Decision Trees, SVMs and Ensemble Learning</H1>
<p1>Andrew J Markland, ajm259@uakron.edu</p1> <br>
<p1>The University of Akron: CEPS : School of Computer Science</p1><br>
<p1>Applied Machine Learning: CPSC:436:010</p1><br>
<p1>Dr. Zhong-Hui Duan </p1><br><br>
<p1>Abstract: The purpose of this assingment is to display understanding and untilization of Decesion tree, ensemble learning</p1><br>
<p1>and SVM's through predicting the likelyhood of strokes through models that we will train with data retrived from NHANES</p1><br><br>
<p1>Disclaimer** While this project is public in my public repository on github, I do not discourage others from using it as a </p1><br>
<p1>reference for their own implementations, but the work written here is expressly written and owned by Andrew J Markland.</p1><br>
<p1>I do not condone nor tolerate academic dishonesty, and should the question of plagerism arise derived from my works</p1><br>
<p1>copies of this repository shall be promptly submitted to the assigning professor and the department of student affairs.</p1>

In [103]:
#includes
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.impute import KNNImputer

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [78]:
"""
    Data Manager
    This cell will handle partitioning and cleaning the data and provide a hand-full of useful
    variables that I can call on to test different data sets based on a partition of the test data, this is primarily 
    going to be used to streamline the testing phases and make data balancing and acquistion faster. 
"""
class DataWrapper:
    def __init__(self):
        self.train_x = None
        self.train_y = None
        self.test_x = None
        self.test_y = None



initialDataPD = pd.read_csv("NHANES_data_stroke_train.csv")

def partitionAndBalance(dataObject):
    global initialDataPD
    X = initialDataPD.drop('stroke', axis=1)
    y = initialDataPD['stroke']
    dataObject.train_x, dataObject.test_x, dataObject.train_y, dataObject.test_y = train_test_split(X, y, test_size = 0.2)
    balanceTrainData(dataObject)
    imputeMissingTestValues(dataObject)
    #print(dataObject.train_X)
    return dataObject
    
    
def balanceTrainData(dataObject):
    combinedDataDF = pd.DataFrame(dataObject.train_x)
    combinedDataDF['stroke'] = dataObject.train_y
    """partition based on classes"""
    strokeDF = combinedDataDF[combinedDataDF['stroke'] == 1]
    noStrokeDF = combinedDataDF[combinedDataDF['stroke'] == 2]
    """clean up missing values"""
    noStrokeDF = noStrokeDF.dropna()
    imputer = KNNImputer(n_neighbors=3)
    imputedDF = imputer.fit_transform(strokeDF)
    cleanStrokeDF = pd.DataFrame(imputedDF, columns=strokeDF.columns)
    noStrokeDF = noStrokeDF.sample(n=len(cleanStrokeDF))
    balancedDF = pd.concat([noStrokeDF, cleanStrokeDF])
    dataObject.train_x = balancedDF.drop('stroke', axis=1)
    dataObject.train_y = balancedDF['stroke']
    
def imputeMissingTestValues(dataObject):
    imputer = KNNImputer(n_neighbors=3)
    imputedTestDF = imputer.fit_transform(dataObject.test_x)
    dataObject.test_x = pd.DataFrame(imputedTestDF, columns=dataObject.test_x.columns)


       SEQN  Income  Sex   Age  Race       Edu  Diastolic    Systolic  \
0    4592.0    0.40  2.0  63.0   1.0  2.000000  76.000000  127.333333   
1    2025.0    0.86  2.0  66.0   7.0  3.000000  81.666667  125.666667   
2    4997.0    2.64  2.0  36.0   1.0  1.000000  62.666667  105.333333   
3    3866.0    2.65  2.0  73.0   3.0  1.666667  85.666667  121.000000   
4    3704.0    4.25  2.0  65.0   3.0  5.000000  76.000000  115.333333   
..      ...     ...  ...   ...   ...       ...        ...         ...   
814  1315.0    5.00  2.0  60.0   3.0  4.000000  59.333333  101.333333   
815  2718.0    0.74  2.0  44.0   2.0  3.000000  80.000000  136.666667   
816  2501.0    3.39  1.0  63.0   6.0  4.000000  73.333333  154.000000   
817  3993.0    2.45  1.0  48.0   4.0  3.000000  89.333333  150.333333   
818  4863.0    4.21  1.0  71.0   2.0  5.000000  68.000000  158.000000   

         Pulse   BMI   HDL   Trig    LDL  TCHOL  kidneys_eGFR  Diabetes  \
0    80.000000  38.3  47.0  145.0  127.0  203.0 

In [101]:
"""
Random Forest run,
    This cell is the testing of the random forest model
"""

#getting a random distribution of data and balancing it
rf_dataObject = DataWrapper()
rf_dataObject = partitionAndBalance(rf_dataObject)

# initializing the random forest
randForest = RandomForestClassifier(n_estimators = 100)

randForest.fit(rf_dataObject.train_x, rf_dataObject.train_y)

prediction = randForest.predict(rf_dataObject.test_x)
accuracy = accuracy_score(rf_dataObject.test_y, prediction)
print(accuracy)

0.7350427350427351
